# Enhancing a Pairs Trading strategy with the application of Machine Learning

Pairs trading is a popular trading strategy in the financial markets that involves identifying two stocks that are highly correlated and trading them in a way that captures the difference in their prices. The goal is to profit from the price convergence of the two stocks while minimizing exposure to market risk. However, finding suitable pairs of stocks for pairs trading can be a daunting task, especially in markets with a large number of securities.

In this context, the framework for pairs selection consist on three different parts: **Dimensionality reduction**, **unsupervised learning** using **OPTICS**, and pairs selection criteria based on **cointegration**, **Hurst exponent**, **half-life**, and **mean zero-crossing**.

- **Dimensionality reduction** techniques, such as principal component analysis (PCA), can be used to reduce the high-dimensional space of financial data to a lower-dimensional space that captures the most important features of the data.
- **Unsupervised learning** algorithms, such as OPTICS, can then be used to cluster the reduced data points into groups based on their similarity, which can aid in identifying pairs of stocks that exhibit similar behavior. 
- Finally, **pairs selection** criteria based on cointegration, Hurst exponent, half-life, and mean zero-crossing can be used to further filter the pairs and identify those that are most suitable for pairs trading.

This project is based on the following paper:

Sarmento, S. M., & Horta, N. (2020). Enhancing a pairs trading strategy with the application of machine learning. Expert Systems with Applications, 158, 113490.


## Universe and Subset Selection Criteria

To implement the pairs trading strategy, we need to select a universe of stocks and filter them based on certain criteria. For this purpose, we retrieve data from an API and select US listed stocks with a market capitalization larger than 300MM and an average volume of over 0.5M.

Next, we select the top 500 companies based on their market capitalization to form a subset of the universe. This is done to reduce the computational complexity of the algorithm and improve its efficiency.

Finally, we set a time frame of 10 years, from March 2013 to March 2023, to analyze the selected stocks and identify suitable pairs for pairs trading.

By applying these universe and subset selection criteria, we ensure that the stocks we analyze are actively traded and have sufficient liquidity to support our trading strategy. Additionally, by focusing on the top companies by market capitalization, we increase the likelihood of identifying highly correlated stocks with similar behavior, which is critical for the success of the pairs trading strategy.

In [1]:
# Libs
import os
from src.functions.package import * # Custom functions 
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.colors as col
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

In [2]:
# Load securities universe and subset based on market cap and volume
universeDF = pd.read_feather('data/processed/universe.feather')
subsetDF = universeDF.sort_values(by='Market Cap',ascending=False).iloc[:499,:]
subsetDF = subsetDF.loc[subsetDF.Volume > 0.5,:]

In [3]:
# Visualize how many stocks do we have within each sector
sectoOv = subsetDF.groupby(by = 'Sector').agg(securities=pd.NamedAgg('Ticker','count'), marketCap=pd.NamedAgg('Market Cap','mean'), volume=pd.NamedAgg('Volume','mean'))
sectoOv = sectoOv.reset_index(drop=False)

# Format variables in order to be displayed
sectoOv['marketCap_formatted'] = '$' + sectoOv['marketCap'].apply(lambda x: '{:,.2f}'.format(x)) + 'B'
sectoOv['volume_formatted'] = sectoOv['volume'].apply(lambda x: '{:,.2f}'.format(x)) + 'M'

# Sort the sector in asceding order
sectoOv = sectoOv.sort_values('securities',ascending=True)

In [4]:
# create the bar trace
bar_trace = go.Bar(
    x=sectoOv['securities'],
    y=sectoOv['Sector'],
    orientation='h',
    marker={'color': sectoOv['securities'], 'colorscale': 'Magma'},
    hovertemplate=[f'Sector: {sect}<br>Securities: {sec}<br>Market Cap: {mc}<br>Volume: {v} units' for sect, sec, mc, v in zip(sectoOv['Sector'], sectoOv['securities'], sectoOv['marketCap_formatted'], sectoOv['volume_formatted'])],
    name=''
)

# create the layout
layout = go.Layout(
    title={
    'text':'<b>Number of Securities by Sector<b><br><sup>Largest 500 securities listed based on market capitalization with over 0.5M average volume</sup>',
    'font':{'size':24,'family':'Goldman Sans','color':'black'}
    },
    xaxis={
    'title':{
    'text':'<b>Number of Securities<b>',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'} 
    },
    yaxis={
    'title':{
    'text':'<b>Sector<b>',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'}
    },
    hoverlabel={
    'font':{'size':15, 'family':'Goldman Sans'}
    },
    plot_bgcolor='white',
    autosize=True,
    height=500
)

# create the figure
fig = go.Figure(data=[bar_trace], layout=layout)

# show the plot
fig.show()

In [5]:
# Visualize how many stocks do we have within each industry
industryOv = subsetDF.groupby(by = 'Industry').agg(securities=pd.NamedAgg('Ticker','count'), marketCap=pd.NamedAgg('Market Cap','mean'), volume=pd.NamedAgg('Volume','mean'))
industryOv = industryOv.reset_index(drop=False)

# Format variables in order to be displayed
industryOv['marketCap_formatted'] = '$' + industryOv['marketCap'].apply(lambda x: '{:,.2f}'.format(x)) + 'B'
industryOv['volume_formatted'] = industryOv['volume'].apply(lambda x: '{:,.2f}'.format(x)) + 'M'

# Sort the sector in asceding order
industryOv = industryOv.sort_values('securities',ascending=True)

In [6]:
# create the bar trace
bar_trace = go.Bar(
    x=industryOv['securities'],
    y=industryOv['Industry'],
    orientation='h',
    marker={'color': industryOv['securities'], 'colorscale': 'Magma'},
    hovertemplate=[f'Industry: {ind}<br>Securities: {sec}<br>Market Cap: {mc}<br>Volume: {v} units' for ind, sec, mc, v in zip(industryOv['Industry'], industryOv['securities'], industryOv['marketCap_formatted'], industryOv['volume_formatted'])],
    name=''
)

# create the layout
layout = go.Layout(
    title={
    'text':'<b>Number of Securities by Industry<b><br><sup>Largest 500 securities listed based on market capitalization with over 0.5M average volume</sup>',
    'font':{'size':24,'family':'Goldman Sans','color':'black'}
    },
    xaxis={
    'title':{
    'text':'<b>Number of Securities<b>',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'} 
    },
    yaxis={
    'title':{
    'text':'<b>Industry<b>',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'}
    },
    hoverlabel={
    'font':{'size':15, 'family':'Goldman Sans'}
    },
    plot_bgcolor='white',
    autosize=True,
    height=500
)

# create the figure
fig = go.Figure(data=[bar_trace], layout=layout)

# show the plot
fig.show()

## Section A: Dimensionality reduction
The first step towards implementing the pairs trading strategy is to find a compact representation for each asset, starting from its price series. The application of Principal Component Analysis (PCA) is proposed for this purpose.

PCA is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of linearly uncorrelated variables, the principal components. Each component can be seen as representing a risk factor. We suggest the application of PCA in the normalized return series, defined as:

$$R_{i,t} = \frac{P_{i,t}-P_{i,t-1}}{P_{i,t-1}}$$

Where $P_{i,t}$ is the price series of a asset i. Using the price series could result in the detection of spurious correlations due to underlying time trends. The number of principal components used defines the number of features for each asset representation.

In [7]:
# Load prices for each asset
dataPrice = pd.read_feather('data/external/subsetprices.feather')
dataPrice = dataPrice[['Date','Adj Close','stock']].pivot_table(values='Adj Close',index='Date',columns='stock')

# Transform prices into daily returns
dataRets = dataPrice.pct_change(1).iloc[1:,]
dataRets.drop(dataRets.columns[dataRets.isna().mean() * 100 > 0],axis=1,inplace=True)
securities = dataRets.columns.to_list()

# Given that we can oly apply PCA to series without missing values, we dropped the seccurities with na
print(f'We are using {len(securities)} securities for the PCA')

We are using 421 securities for the PCA


In [8]:
# PCA with 5 components
pca = PCA(5)
transfDF = pca.fit_transform(dataRets)
compsDF = pd.DataFrame(pca.components_)

In [9]:
# Explained variance based on the number of components chosen
expVarRatio = pd.DataFrame({
    'comps': ['C'+str(i+1) for i in range(len(pca.explained_variance_ratio_)+1)],
    'val': np.append(pca.explained_variance_ratio_,np.sum(pca.explained_variance_ratio_)),
    'measure': ['relative' for i in range(len(pca.explained_variance_ratio_)+1)]
})

expVarRatio.iloc[-1,0] = 'Total'
expVarRatio.iloc[-1,2] = 'total' 

# Calculate the cumulative percentage
expVarRatio['cumulative'] = expVarRatio['val'].cumsum()

# Fortmat labels
expVarRatio['val_formatted'] = expVarRatio['val'].apply(lambda x: '{:,.2f}'.format(x*100)+'%')
expVarRatio['cumPct_formatted'] = expVarRatio['cumulative'].apply(lambda x: '{:,.2f}'.format(x*100)+'%')

# Plot the explained variance by the components
water_trace = go.Waterfall(
    orientation='v',
    measure=expVarRatio['measure'],
    x=expVarRatio['comps'],
    y=expVarRatio['val'],
    decreasing={"marker":{"color":"#75160C"}},
    increasing={"marker":{"color":"#094536"}},
    totals={"marker":{"color":"#2A3F4D"}},
    hovertemplate=[f'Component: {comp}<br>Exp. var: {ex1}' for comp, ex1 in zip(expVarRatio['comps'], expVarRatio['val_formatted'])],
    name=''
)

# create the layout
layout = go.Layout(
    title={
    'text':'<b>Explained variance ratio<b><br><sup>Percentage of variance that is attributed by each of the selected components</sup>',
    'font':{'size':24,'family':'Goldman Sans','color':'black'}
    },
    xaxis={
    'title':{
    'text':'<b>Components<b>',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'} 
    },
    yaxis={
    'title':{
    'text':'<b>Explained Variance<b>',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickformat':'.2%',
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'}
    },
    hoverlabel={
    'font':{'size':15, 'family':'Goldman Sans'}
    },
    plot_bgcolor='white',
    autosize=True,
    height=500
)

# create the figure
fig = go.Figure(data=[water_trace], layout=layout)

# show the plot
fig.show()

In [10]:
# Visualize the securities using the first two components of the PCA
pcaDF = compsDF.copy()
pcaDF.columns = ['comp'+str(i+1) for i in range(len(pcaDF.columns))]
pcaDF['securities'] = securities
pcaDF = pd.merge(pcaDF,universeDF[['Ticker','Company','Sector','Industry','Market Cap']],how='left',left_on='securities',right_on='Ticker')
pcaDF['colors'] = pd.factorize(pcaDF['Sector'])[0] + 1

# Plot scatter plot
scatter_trace = go.Scatter(
    x=pcaDF['comp1'],
    y=pcaDF['comp2'],
    mode='markers',
    marker={'color': pcaDF['colors'], 'colorscale': 'Magma'},
    hovertemplate=[f'Sector: {sect}<br>Company: {comp}<br>Symbol: {tick}<br>Market Cap: ${mkc}B' for sect, comp, tick, mkc in zip(pcaDF['Sector'], pcaDF['Company'], pcaDF['securities'], pcaDF['Market Cap'])],
    name=''
)

# create the layout
layout = go.Layout(
    title={
    'text':'<b>Exploring Sector-wise Clustering of Stocks<b><br><sup>Using Principal Component Analysis (PCA)</sup>',
    'font':{'size':24,'family':'Goldman Sans','color':'black'}
    },
    xaxis={
    'title':{
    'text':'<b>Comp. 1<b>',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickformat':'.2f',
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'} 
    },
    yaxis={
    'title':{
    'text':'<b>Comp. 2<b>',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickformat':'.2f',
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'}    
    },
    hoverlabel={
    'font':{'size':15, 'family':'Goldman Sans'}
    },    
    plot_bgcolor='white',
    autosize=True,
    height=500
)

# create the figure
fig = go.Figure(data=[scatter_trace], layout=layout)

# show the plot
fig.show()

## Section B: Unsupervised learning

We explore the application of clustering techniques for asset grouping based on a compact representation, it becomes crucial to select an appropriate algorithm that meets specific requirements like: 
- Avoid specifying the number of clusters in advance. 
- Not mandatory to group all securities. 
- Accounting for outliers. 
- Avoiding assumptions regarding the clusters' shape.

 Without strict assignment, the number of combinations when looking for pairs would increase, undermining the initial objective. By making the number of clusters data-driven, we minimize bias. Outliers should not be incorporated into the clusters, and grouping all assets should not be enforced. Finally, as prior information does not indicate that clusters should be regularly shaped, the selected algorithm should avoid this assumption.

In light of these requirements, a density-based clustering algorithm appears to be an appropriate choice. This algorithm forms clusters with arbitrary shapes, removing the need to adopt any gaussianity assumptions, and is naturally robust to outliers by not grouping every point in the data set. Furthermore, it does not require a specific number of clusters to be specified.

The OPTICS algorithm, offers an enhanced solution to this problem. Based on DBSCAN, it introduces important concepts that allow for varying ε implementation. In this setting, the investor only needs to specify the minimum number of points to form a cluster, as the algorithm is capable of detecting the most appropriate ε for each cluster. Thus, the use of OPTICS is proposed not only to account for varying cluster densities but also to simplify the investor's task. By delving into the details of the OPTICS algorithm and its enhanced features, we can better understand its potential for asset grouping and its contribution to the field of data analysis.

In [14]:
# Run the OPTICS algorithm based on the compact representation of the securities
optics = Optics(min_samples=5,max_eps=0.3,dist_metric='chebyshev',stocksNames=securities, n_jobs=-1)
securitiesClusters = optics.fit(compsDF)

Clusters discovered: 16


In [15]:
# Create a dataframe for the clusters
clustersDF = securitiesClusters.copy()
clustersDF = clustersDF.reset_index(drop=False)
clustersDF.columns = ['securities','clusters']

# Select the securities listed in the clusters
clustersRets = dataRets[clustersDF['securities']]

# Reduce the dimentionality in order to plot it
pca = PCA(3)
transfDF = pca.fit_transform(clustersRets)
compsDF = pd.DataFrame(pca.components_)
compsDF.columns = ['comp'+str(i+1) for i in range(len(compsDF.columns))]
compsDF = pd.concat([clustersDF,compsDF],axis=1,ignore_index=False)
compsDF = pd.merge(compsDF,universeDF[['Ticker','Company','Sector','Industry','Market Cap']],how='left',left_on='securities',right_on='Ticker')

In [16]:
# Plot scatter plot
scatter3d_trace = go.Scatter3d(
    x=compsDF['comp1'],
    y=compsDF['comp2'],
    z=compsDF['comp3'],
    mode='markers',
    marker={'color': compsDF['clusters'], 'colorscale': 'Magma', 'opacity':0.8},
    hovertemplate=[f'Cluster: {cls}<br>Company: {comp}<br>Symbol: {sec}' for cls, comp, sec in zip(compsDF['clusters'], compsDF['Company'], compsDF['securities'])],
    name=''
)

# create the layout
layout = go.Layout(
    title={
    'text':'<b>Exploring Unsupervised-wise Clustering of Stocks<b><br><sup>Using OPTICS</sup>',
    'font':{'size':24,'family':'Goldman Sans','color':'black'}
    },
    scene={
    'xaxis':{
    'title':{
    'text':'Comp. 1',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickformat':'.2f',
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'}     
    },
    'yaxis':{
    'title':{
    'text':'Comp. 2',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickformat':'.2f',
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'}     
    },
    'zaxis':{
    'title':{
    'text':'Comp. 3',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickformat':'.2f',
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'}     
    }
    },
    hoverlabel={
    'font':{'size':15, 'family':'Goldman Sans'}
    },    
    plot_bgcolor='white',
    autosize=True,
    margin={
    'l':5,
    'r':5,
    'b':5,
    't':55
    },
    height=500       
)


# create the figure
fig = go.Figure(data=[scatter3d_trace], layout=layout)

# show the plot
fig.show()

## Section C: Pairs selection criteria
After generating clusters of assets, selecting pairs to trade requires the definition of a set of rules. It is crucial to maintain the equilibrium of pairs. To achieve this, the unification of methods used in separate research work is proposed. A pair is selected only if it satisfies four conditions. 
1. Both securities forming the pair must be cointegrated, which is tested using the Engle-Granger test.
2. The pair's spread Hurst exponent must be less than 0.5 to provide more confidence in the mean-reversion character of the spread.
3. Pairs with extreme values of half-life are filtered out to ensure coherence between mean-reversion time and trading period.
4. Every spread must cross its mean at least 3 times per year, ensuring an average of one trade per month.


In [17]:
# Get Pairs
getpairs = getPairs(alpha=0.10, hurstThresh=0.5, minHLife=22, maxHLife=132, zeroCrosses=3)
pairs = getpairs.eval(clusters=securitiesClusters,prices=dataPrice)

In [18]:
# Visualize how each criteria reduce the pairs universe
pairsOps = pd.DataFrame.from_dict({'keys':getpairs.universe},orient='columns')
pairsOps = pairsOps.reset_index(drop=False).sort_values('keys',ascending=False)
pairsOps['index'] = pairsOps['index'].apply(lambda x: x.capitalize())
pairsOps['delta'] = pairsOps['keys'] - pairsOps['keys'].shift(-1)
pairsOps['delta'] = pairsOps['delta'].shift(1)
pairsOps['delta'] = pairsOps['delta']*-1
pairsOps.iloc[0,0] = 'Pot. Pairs'
pairsOps.iloc[0,2] = pairsOps.iloc[0,1]
pairsOps['measure'] = 'relative'
pairsOps.iloc[-1,3] = 'total'

# Plot the explained variance by the components
water_trace = go.Waterfall(
    orientation='v',
    measure=pairsOps['measure'],
    x=pairsOps['index'],
    y=pairsOps['delta'],
    decreasing={"marker":{"color":"#75160C"}},
    increasing={"marker":{"color":"#094536"}},
    totals={"marker":{"color":"#2A3F4D"}},
    hovertemplate=[f'Pairs: {npairs}<br>Delta: {delta}' for npairs, delta in zip(pairsOps['keys'],pairsOps['delta'])],
    name=''
)

# create the layout
layout = go.Layout(
    title={
    'text':'<b>Pairs selection funnel<b><br><sup>A pair is selected only if it satisfies four conditions</sup>',
    'font':{'size':24,'family':'Goldman Sans','color':'black'}
    },
    xaxis={
    'title':{
    'text':'<b>Criterias<b>',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'} 
    },
    yaxis={
    'title':{
    'text':'<b>Number of pairs<b>',    
    'font':{'size':20,'family':'Goldman Sans','color':'black'}
    },
    'tickformat':'.f',
    'tickfont':{'size':16,'family':'Goldman Sans','color':'black'}
    },
    hoverlabel={
    'font':{'size':15, 'family':'Goldman Sans'}
    },
    plot_bgcolor='white',
    autosize=True,
    height=500
)


# create the figure
fig = go.Figure(data=[water_trace], layout=layout)

# show the plot
fig.show()